In [6]:
# how to install
!pip install xlrd==1.2.0
!apt-get update
!apt-get install -y alien
!alien -i /workspace/ailab/ailab-mgmt/packages/*.rpm
!apt-get install libaio1
!export ORACLE_HOME='/usr/lib/oracle/21/client64'
!export LD_LIBRARY_PATH='/usr/lib/oracle/21/client64/lib/:$LD_LIBRARY_PATH'
!python -m pip install cx_Oracle --upgrade
!pip install sqlalchemy
!pip install sshtunnel

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [1479 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2450 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1161 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1990 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [883 kB]
Fetched 8297 kB in 3s (2521 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will 

In [7]:
import datetime
import logging
import getpass
import cx_Oracle
import pandas as pd
from sshtunnel import SSHTunnelForwarder

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def tunnel_open(target_host, service_name, 
                bastion = 'EUPOKTOCI01.ent.bhicorp.com', target_port=1521, 
                ssh_username="", ssh_password="", 
                USERNAME='SSO502826724', PASSWORD='Ailab27ailab',
                close_connection = False
                   ):
    try:
        server = SSHTunnelForwarder(
            bastion,
            ssh_username=ssh_username,
            ssh_password=ssh_password,
            remote_bind_address=(target_host, target_port)
        )
        print("checking OKTA...")
        server.start()

        port = server.local_bind_port
        dsn_tns = cx_Oracle.makedsn('localhost', port, service_name=service_name)

        connection = cx_Oracle.connect(USERNAME, PASSWORD, dsn_tns)

        print("reading data on port:", port, service_name)  # show assigned local port

        if close_connection:
            server.stop()
            print("server stopped")
        else:
            print("left the server open")
        return connection
    
    except Exception as e:
        print(e)
        server.stop()
        
        

In [22]:
_db='argo'
_db='devargo'

if _db=='argo':
    _host = 'argoexa-vip1.int.bkrh.oracleoutsourcing.com'
    _username = 'SSO502826724'
    _password = 'W3Lc0m3_1p'
    
elif _db=='devargo':
        
    _host='nonprod-argoexa-vip1.int.bkrh.oracleoutsourcing.com'
    _username = 'TPS_PROJECTS_DM'
    _password = 'Oci_tps12dms'

connection = tunnel_open(target_host = _host, 
                         service_name = _db, 
                         ssh_username= input('BASTION USERNAME (SSO):'), 
                         ssh_password= getpass.getpass('BASTION PASSWORD (SSO):'),
                         USERNAME=_username, 
                         PASSWORD=_password,
                         close_connection = False)

BASTION USERNAME (SSO): 212442540
BASTION PASSWORD (SSO): ··········


checking OKTA...
reading data on port: 44345 devargo
left the server open


In [23]:

_type='similarity'
_table='TPS_PROJECTS_DM.AK_TO_BHLSE_SIMILARITY_FEEDBACK'

#_type='cluster'
#_table='TPS_PROJECTS_DM.AK_TO_BHLSE_SIMILARITY_CLUSTER_FEEDBACK'

sql = f"SELECT * FROM {_table} WHERE CREATED_BY NOT IN ('cappant', '503273934', '503189761', '212779015')"




days = 600

In [24]:
def read_sql(sql, connection):
    df = pd.read_sql(sql, con=connection)
    logging.info("Loaded data with query: {}".format(sql))
    
    return df

In [25]:
df = read_sql(sql, connection)

df

,ID,SOURCE_CASE_NUMBER,DEST_CASE_NUMBER,FEEDBACK_VALUE,CREATION_DATE,CREATED_BY,START_PROCESSED_DATE,PROCESSED_DATE,PROCESSED
0,53,45c3216a-60bc-48e3-aae4-afe6202ac63f,90399001,OK,2021-12-02 08:05:02,105027817,None,None,None
1,54,45c3216a-60bc-48e3-aae4-afe6202ac63f,90378743,DISPUTED,2021-12-02 08:05:07,105027817,None,None,None
2,55,45c3216a-60bc-48e3-aae4-afe6202ac63f,90411192,OK,2021-12-02 08:05:14,105027817,None,None,None
3,56,45c3216a-60bc-48e3-aae4-afe6202ac63f,90397135,OK,2021-12-02 08:06:50,105027817,None,None,None
4,57,45c3216a-60bc-48e3-aae4-afe6202ac63f,90393628,OK,2021-12-02 08:07:28,105027817,None,None,None
...,...,...,...,...,...,...,...,...,...
405,238,90295272,90383003,KO,2021-12-06 12:56:38,105710464,None,None,None
406,326,12ddf117-538c-4db2-88d1-0a7c3942576e,90381097,KO,2021-12-07 10:30:28,105046213,None,None,None
407,327,56f0ebc8-3d76-426c-ac26-48f3c3798113,90415360,KO,2021-12-07 10:50:07,105046213,None,None,None
408,328,56f0ebc8-3d76-426c-ac26-48f3c3798113,90391245,DISPUTED,2021-12-07 10:50:14,105046213,None,None,None


In [26]:
from datetime import datetime

now = datetime.now() # current date and time
date_time = now.strftime("%Y%m%d-%H%M%S")
df.to_csv(f'../data/data_{_type}_{_db}_feedback_{date_time}.csv')

In [27]:
connection.close()